The order of methods in code is different from in paper. See comment for details.

Method 5 requires the following lshashpy3 package. Need to install before use.

In [1]:
# pip install lshashpy3

In [2]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.cluster import KMeans
import time
from lshashpy3 import LSHash

In [3]:
(train_X, train_y), (test_X, test_y) = tf.keras.datasets.mnist.load_data()
train_X = train_X.reshape(train_X.shape[0], train_X.shape[1]*train_X.shape[2])
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1]*test_X.shape[2])

In [4]:
# separate train into 10 labels
train_X_buckets = []
for i in range(10):
    train_X_buckets.append(train_X[train_y == i])

In [5]:
train_X.shape, test_X.shape

((60000, 784), (10000, 784))

In [6]:
# Using all of training set
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(train_X, train_y)


KNeighborsClassifier(n_neighbors=1)

In [7]:
neigh.score(test_X, test_y)

0.9691

In [8]:
M = 10000

In [22]:
# Baseline: random selection
# return accuracy, elapsed time
def baseline(M):
    np.random.seed(0)
    random_M_indices = np.random.choice(train_X.shape[0], M, replace=False)
    train_X_base = train_X[random_M_indices]
    train_y_base = train_y[random_M_indices]
    neigh_base = KNeighborsClassifier(n_neighbors=1)
    neigh_base.fit(train_X_base, train_y_base)
    start = time.time()
    acc = neigh_base.score(test_X, test_y)
    elapse = time.time() - start
    return acc, elapse

In [ ]:
baseline(M)

In [ ]:
train_y

In [ ]:
# 1. (Method 3 in paper) KMeans with 10 buckets for 10 labels

In [1]:
print(np.sum(list(map(lambda x: len(x), train_X_buckets))))
assert(np.sum(list(map(lambda x: len(x), train_X_buckets))) == train_X.shape[0])

NameError: name 'np' is not defined

In [ ]:
for i in range(10):
    print(train_X_buckets[i].shape)

In [ ]:
cluster_center_bucket = []
for i in range(10):
    kmeans = KMeans(n_clusters=int(M/10), random_state=0, n_init="auto").fit(train_X_buckets[i])
    cluster_center_bucket.append(kmeans.cluster_centers_)

In [ ]:
cluster_center_bucket[0].shape

In [ ]:
np.concatenate(cluster_center_bucket, axis=0).shape

In [ ]:
train_X_kmeans_bucket = np.concatenate(cluster_center_bucket, axis=0)

In [ ]:
train_y_kmeans_bucket = []
for i in range(10):
    train_y_kmeans_bucket.append([i] * cluster_center_bucket[i].shape[0])
train_y_kmeans_bucket = np.concatenate(train_y_kmeans_bucket)
train_y_kmeans_bucket

In [ ]:
print(train_y_kmeans_bucket[0], train_y_kmeans_bucket[1000], train_y_kmeans_bucket[2000])
train_y_kmeans_bucket.shape

In [ ]:
assert(train_X_kmeans_bucket.shape[0] == M)
assert(train_y_kmeans_bucket.shape[0] == M)

In [ ]:
neigh_kmeans_bucket = KNeighborsClassifier(n_neighbors=1)
neigh_kmeans_bucket.fit(train_X_kmeans_bucket, train_y_kmeans_bucket)

In [ ]:
neigh_kmeans_bucket.score(test_X, test_y)

In [ ]:
# 2. (Method 4 in paper) KMeans cluster center is the mean of all points in the cluster, which doesn't exist in the original dataset.
# What if we find the 1NN of the cluster center so that we return a subset of the training data?

In [ ]:
center_1nn_bucket = []
neigh_center = KNeighborsClassifier(n_neighbors=1)
neigh_center.fit(train_X, train_y)

In [ ]:
train_X[np.concatenate(neigh_center.kneighbors(cluster_center_bucket[0], return_distance=False))].shape

In [ ]:
# use the cluster centers as queries to find the closest points in raw training dataset
for i in range(10):
    center_1nn_bucket.append(train_X[np.concatenate(neigh_center.kneighbors(cluster_center_bucket[i], return_distance=False))])

In [ ]:
train_X_kmeans_1nn_bucket = np.concatenate(center_1nn_bucket, axis=0)
train_y_kmeans_1nn_bucket = train_y_kmeans_bucket
assert(train_X_kmeans_1nn_bucket.shape[0] == M)
assert(train_y_kmeans_1nn_bucket.shape[0] == M)
neigh_kmeans_1nn_bucket = KNeighborsClassifier(n_neighbors=1)
neigh_kmeans_1nn_bucket.fit(train_X_kmeans_1nn_bucket, train_y_kmeans_1nn_bucket)

In [ ]:
neigh_kmeans_1nn_bucket.score(test_X, test_y)

In [ ]:
# 3. (Method 1 in paper) KMeans but doesn't separate into 10 buckets. Use majority vote in clusters
# It is possible that an outlier in the bucket labelled "2" gets grouped into a single point cluster by the KMeans Clustering
# Then a query near it the centroid will be labeled "2" even if every point around it is "1".

In [ ]:
kmeans = KMeans(n_clusters=M, random_state=0, n_init="auto").fit(train_X)

In [ ]:
kmeans.labels_

In [ ]:
clusters = []
train_X_majority = []
train_y_majority = []

for _ in range(M):
    clusters.append([])
for i in range(train_X.shape[0]):
    clusters[kmeans.labels_[i]].append(i)
for clust in clusters:
    # clust contains indices of training data
    count = [0] * 10
    for j in clust:
        count[train_y[j]] += 1
    label = count.index(max(count)) # Majority vote. Break ties by the smallest label
    centroid = np.mean(train_X[clust], axis=0)
    train_X_majority.append(centroid)
    train_y_majority.append(label)

In [ ]:
train_X_majority = np.array(train_X_majority)
train_y_majority = np.array(train_y_majority)
assert(train_X_majority.shape[0] == M)
assert(train_y_majority.shape[0] == M)
neigh_majority_1nn = KNeighborsClassifier(n_neighbors=1)
neigh_majority_1nn.fit(train_X_majority, train_y_majority)

In [ ]:
neigh_majority_1nn.score(test_X, test_y)

In [ ]:
# 4. (Method 2 in paper) Purify the clusters to contain a single label and relocate the cluster centers.
# There're 2 ways to relocate the center. 
# One is to take the mean of the remaining points in the cluster after purification.
# Another is to rerun the KMeans clustering algorithm on the whole set of remaining points.
# In fact, this 2nd way can be seemed as an iterative process. 
# We can iteratively kick out points that are not majority of a cluster and rerun the KMeans algorithm until we cannot kick out any point anymore.
# Then we would have clusters that are very separated with clear boundaries.

# Discussion: the 2nd way will be infeasible to implement because KMeans takes too long (~15min) on the whole MNIST dataset. 
# Can only try to iterate a small number of times.

In [ ]:
clusters = []
train_X_majority_pure = []
train_y_majority_pure = []

for _ in range(M):
    clusters.append([])
for i in range(train_X.shape[0]):
    clusters[kmeans.labels_[i]].append(i)
for clust in clusters:
    # clust contains indices of training data
    count = [0] * 10
    for j in clust:
        count[train_y[j]] += 1
    label = count.index(max(count)) # Majority vote. Break ties by the smallest label
    # only take mean of the majority
    majority = [] # contains indices in training data of the clust majority
    for j in clust:
        if train_y[j] == label:
            majority.append(j)
    centroid = np.mean(train_X[majority], axis=0)
    train_X_majority_pure.append(centroid)
    train_y_majority_pure.append(label)

In [ ]:
train_X_majority_pure = np.array(train_X_majority_pure)
train_y_majority_pure = np.array(train_y_majority_pure)
assert(train_X_majority_pure.shape[0] == M)
assert(train_y_majority_pure.shape[0] == M)
neigh_majority_pure_1nn = KNeighborsClassifier(n_neighbors=1)
neigh_majority_pure_1nn.fit(train_X_majority_pure, train_y_majority_pure)

In [ ]:
neigh_majority_pure_1nn.score(test_X, test_y)

In [ ]:
# 5. Locality Sensitive Hashing

# In LSH, we cannot specify the number of clusters/centroids (M). 
# Therefore, we may need to do the above methods again based on the new M from the LSH hash. 

In [ ]:
# 6. (Didn't implement this one. Can discuss)
# What if we do KMeans multiple times with different random seeds? 
# Since KMeans is random, I suspect averaging over these will give a better performance.
# Then we would have multiple copies of M/10 centroids. 
# What if we do another KMeans clustering over these?

# Do this for the 10 buckets, not the whole dataset, since KMeans on the whole dataset takes too long

In [ ]:
# 7. As Method 2 reasons:
# 'It is possible that an outlier in the bucket labelled "2" gets grouped into a single point cluster by the KMeans Clustering'
# 'Then a query near it the centroid will be labeled "2" even if every point around it is "1".'
# What if we coarsely purify the dataset first in each cluster, and then do the bucket?

In [ ]:
hashtable_center_bucket = []
for _ in range(10):
    hashtable_center_bucket.append([])

k = (M//10).bit_length() # hash size = log2(M/10)
L = 1  # number of tables
d = 784 # Dimension of Feature vector

center_num = 0

for i in range(10):
    lsh = LSHash(hash_size=k, input_dim=d, num_hashtables=L)
    for j in train_X_buckets[i]:
        lsh.index(j)
    hashtable = lsh.hash_tables[0].storage
    center_num += len(list(hashtable.keys()))
    for key, value in hashtable.items():
        cluster = []
        for v in value:
            cluster.append(np.array(v[0]))
        cluster = np.array(cluster)
        # np.mean(cluster, axis = 0).astype(int)
        hashtable_center_bucket[i].append(np.mean(cluster, axis = 0))


In [ ]:
for i in range(10):
    print(len(hashtable_center_bucket[i]))

In [ ]:
center_num

In [ ]:
train_X_lsh_bucket = np.concatenate(hashtable_center_bucket, axis=0)
train_X_lsh_bucket.shape

In [ ]:
baseline(center_num)

In [ ]:
train_y_lsh_bucket = []
for i in range(10):
    train_y_lsh_bucket.append([i] * len(hashtable_center_bucket[i]))
train_y_lsh_bucket = np.concatenate(train_y_lsh_bucket)
len(train_y_lsh_bucket), train_y_lsh_bucket

In [ ]:
neigh_lsh_bucket = KNeighborsClassifier(n_neighbors=1)
neigh_lsh_bucket.fit(train_X_lsh_bucket, train_y_lsh_bucket)
neigh_lsh_bucket.score(test_X, test_y)

In [ ]:
# Summarize all methods

In [9]:
def avgTime(neigh):
    acc, time_sum = 0, 0
    for _ in range(10):
        start = time.time()
        acc = neigh.score(test_X, test_y)
        time_sum += time.time() - start
    return acc, time_sum/10

In [10]:
# Baseline: random selection
# return accuracy, elapsed time
def baseline(M):
    np.random.seed(0)
    random_M_indices = np.random.choice(train_X.shape[0], M, replace=False)
    train_X_base = train_X[random_M_indices]
    train_y_base = train_y[random_M_indices]
    neigh_base = KNeighborsClassifier(n_neighbors=1)
    neigh_base.fit(train_X_base, train_y_base)
    acc, elapse = avgTime(neigh_base)
    return acc, elapse

In [11]:
# summarize method1 into function
# return accuracy, elapsed time
# return cluster_center_bucket, train_y_kmeans_bucket (used for method2)
def method1(M):
    # train_X_buckets = []
    # for i in range(10):
    #     train_X_buckets.append(train_X[train_y == i])
    assert(np.sum(list(map(lambda x: len(x), train_X_buckets))) == train_X.shape[0])
    cluster_center_bucket = []
    for i in range(10):
        kmeans = KMeans(n_clusters=int(M/10), random_state=0, n_init="auto").fit(train_X_buckets[i])
        cluster_center_bucket.append(kmeans.cluster_centers_)
    train_X_kmeans_bucket = np.concatenate(cluster_center_bucket, axis=0)
    train_y_kmeans_bucket = []
    for i in range(10):
        train_y_kmeans_bucket.append([i] * cluster_center_bucket[i].shape[0])
    train_y_kmeans_bucket = np.concatenate(train_y_kmeans_bucket)
    # assert(train_X_kmeans_bucket.shape[0] == M)
    # assert(train_y_kmeans_bucket.shape[0] == M)
    neigh_kmeans_bucket = KNeighborsClassifier(n_neighbors=1)
    neigh_kmeans_bucket.fit(train_X_kmeans_bucket, train_y_kmeans_bucket)
    acc, elapse = avgTime(neigh_kmeans_bucket)
    return acc, elapse, cluster_center_bucket, train_y_kmeans_bucket

In [12]:
# summarize method2 into function
# use cluster_center_bucket, train_y_kmeans_bucket from method1
# return accuracy, elapsed time
def method2(M, cluster_center_bucket, train_y_kmeans_bucket):
    center_1nn_bucket = []
    neigh_center = KNeighborsClassifier(n_neighbors=1)
    neigh_center.fit(train_X, train_y)
    for i in range(10):
        center_1nn_bucket.append(train_X[np.concatenate(neigh_center.kneighbors(cluster_center_bucket[i], return_distance=False))])
    train_X_kmeans_1nn_bucket = np.concatenate(center_1nn_bucket, axis=0)
    train_y_kmeans_1nn_bucket = train_y_kmeans_bucket
    # assert(train_X_kmeans_1nn_bucket.shape[0] == M)
    # assert(train_y_kmeans_1nn_bucket.shape[0] == M)
    neigh_kmeans_1nn_bucket = KNeighborsClassifier(n_neighbors=1)
    neigh_kmeans_1nn_bucket.fit(train_X_kmeans_1nn_bucket, train_y_kmeans_1nn_bucket)
    acc, elapse = avgTime(neigh_kmeans_1nn_bucket)
    return acc, elapse

In [13]:
# summarize method3 into function
# return accuracy and elapsed time
# return the kmeans model (used for method4)
def method3(M):
    kmeans = KMeans(n_clusters=M, random_state=0, n_init="auto").fit(train_X)
    clusters = []
    train_X_majority = []
    train_y_majority = []

    for _ in range(M):
        clusters.append([])
    for i in range(train_X.shape[0]):
        clusters[kmeans.labels_[i]].append(i)
    for clust in clusters:
        # clust contains indices of training data
        count = [0] * 10
        for j in clust:
            count[train_y[j]] += 1
        label = count.index(max(count)) # Majority vote. Break ties by the smallest label
        centroid = np.mean(train_X[clust], axis=0)
        train_X_majority.append(centroid)
        train_y_majority.append(label)
    train_X_majority = np.array(train_X_majority)
    train_y_majority = np.array(train_y_majority)
    # assert(train_X_majority.shape[0] == M)
    # assert(train_y_majority.shape[0] == M)
    neigh_majority_1nn = KNeighborsClassifier(n_neighbors=1)
    neigh_majority_1nn.fit(train_X_majority, train_y_majority)
    acc, elapse = avgTime(neigh_majority_1nn)
    return acc, elapse, kmeans

In [14]:
# summarize method4 into function
# use the kmeans model from method3
# return accuracy and elapsed time
def method4(M, kmeans):
    clusters = []
    train_X_majority_pure = []
    train_y_majority_pure = []

    for _ in range(M):
        clusters.append([])
    for i in range(train_X.shape[0]):
        clusters[kmeans.labels_[i]].append(i)
    for clust in clusters:
        # clust contains indices of training data
        count = [0] * 10
        for j in clust:
            count[train_y[j]] += 1
        label = count.index(max(count)) # Majority vote. Break ties by the smallest label
        # only take mean of the majority
        majority = [] # contains indices in training data of the clust majority
        for j in clust:
            if train_y[j] == label:
                majority.append(j)
        centroid = np.mean(train_X[majority], axis=0)
        train_X_majority_pure.append(centroid)
        train_y_majority_pure.append(label)
    train_X_majority_pure = np.array(train_X_majority_pure)
    train_y_majority_pure = np.array(train_y_majority_pure)
    # assert(train_X_majority_pure.shape[0] == M)
    # assert(train_y_majority_pure.shape[0] == M)
    neigh_majority_pure_1nn = KNeighborsClassifier(n_neighbors=1)
    neigh_majority_pure_1nn.fit(train_X_majority_pure, train_y_majority_pure)
    acc, elapse = avgTime(neigh_majority_pure_1nn)
    return acc, elapse

In [15]:
# summarize method5 into function
# return accuracy, elapsed time
# return a compression number close to M
def method5(M):
    hashtable_center_bucket = []
    for _ in range(10):
        hashtable_center_bucket.append([])

    k = (M//10).bit_length() + 3 # hash size = log2(M/10) + 3
    L = 1  # number of tables
    d = 784 # Dimension of Feature vector

    center_num = 0
    for i in range(10):
        lsh = LSHash(hash_size=k, input_dim=d, num_hashtables=L)
        for j in train_X_buckets[i]:
            lsh.index(j)
        hashtable = lsh.hash_tables[0].storage
        center_num += len(list(hashtable.keys()))
        for key, value in hashtable.items():
            cluster = []
            for v in value:
                cluster.append(np.array(v[0]))
            cluster = np.array(cluster)
            # np.mean(cluster, axis = 0).astype(int)
            hashtable_center_bucket[i].append(np.mean(cluster, axis = 0))

    train_X_lsh_bucket = np.concatenate(hashtable_center_bucket, axis=0)
    train_y_lsh_bucket = []
    for i in range(10):
        train_y_lsh_bucket.append([i] * len(hashtable_center_bucket[i]))
    train_y_lsh_bucket = np.concatenate(train_y_lsh_bucket)
    len(train_y_lsh_bucket), train_y_lsh_bucket
    neigh_lsh_bucket = KNeighborsClassifier(n_neighbors=1)
    neigh_lsh_bucket.fit(train_X_lsh_bucket, train_y_lsh_bucket)
    acc, elapse = avgTime(neigh_lsh_bucket)
    return acc, elapse, center_num

In [16]:
def method7(M):
    kmeans = KMeans(n_clusters=M, random_state=0, n_init="auto").fit(train_X)
    clusters = []
    # train_X_majority_pure = []
    # train_y_majority_pure = []
    kept = []

    for _ in range(M):
        clusters.append([])
    for i in range(train_X.shape[0]):
        clusters[kmeans.labels_[i]].append(i)
    for clust in clusters:
        # clust contains indices of training data
        count = [0] * 10
        for j in clust:
            count[train_y[j]] += 1
        label = count.index(max(count)) # Majority vote. Break ties by the smallest label
        # only take mean of the majority
        majority = [] # contains indices in training data of the clust majority
        for j in clust:
            if train_y[j] == label:
                majority.append(j)
                kept.append(j)
        # centroid = np.mean(train_X[majority], axis=0)
        # train_X_majority_pure.append(centroid)
        # train_y_majority_pure.append(label)
    train_X_pure = train_X[kept]
    train_y_pure = train_y[kept]

    train_X_buckets = []
    for i in range(10):
        train_X_buckets.append(train_X_pure[train_y_pure == i])
    # assert(np.sum(list(map(lambda x: len(x), train_X_buckets))) == train_X.shape[0])
    cluster_center_bucket = []
    for i in range(10):
        kmeans = KMeans(n_clusters=int(M/10), random_state=0, n_init="auto").fit(train_X_buckets[i])
        cluster_center_bucket.append(kmeans.cluster_centers_)
    train_X_kmeans_bucket = np.concatenate(cluster_center_bucket, axis=0)
    train_y_kmeans_bucket = []
    for i in range(10):
        train_y_kmeans_bucket.append([i] * cluster_center_bucket[i].shape[0])
    train_y_kmeans_bucket = np.concatenate(train_y_kmeans_bucket)
    neigh_kmeans_bucket = KNeighborsClassifier(n_neighbors=1)
    neigh_kmeans_bucket.fit(train_X_kmeans_bucket, train_y_kmeans_bucket)
    acc, elapse = avgTime(neigh_kmeans_bucket)
    
    # neigh_majority_pure_1nn = KNeighborsClassifier(n_neighbors=1)
    # neigh_majority_pure_1nn.fit(train_X_majority_pure, train_y_majority_pure)
    # acc, elapse = avgTime(neigh_majority_pure_1nn)
    return acc, elapse

In [17]:
M = 10000

In [18]:
# separate train into 10 labels
train_X_buckets = []
for i in range(10):
    train_X_buckets.append(train_X[train_y == i])

In [19]:
acc7, elapse7 = method7(M)

In [20]:
acc7, elapse7

(0.9676, 2.479336667060852)

In [65]:
accB, elapseB = baseline(M)
accB, elapseB

(0.9492, 2.855719208717346)

In [21]:
acc1, elapse1, cluster_center_bucket, train_y_kmeans_bucket = method1(M)
acc1, elapse1

(0.9708, 2.4796133279800414)

In [67]:
acc2, elapse2 = method2(M, cluster_center_bucket, train_y_kmeans_bucket)
acc2, elapse2

(0.9568, 2.803221011161804)

In [68]:
acc3, elapse3, kmeans = method3(M)
acc3, elapse3

(0.9638, 2.770078206062317)

In [69]:
acc4, elapse4 = method4(M, kmeans)
acc4, elapse4

(0.9678, 2.7765588045120237)

In [70]:
# Do method6 multiple times due to randomness
# obtain a new M
accList, elapseList, newMList = [], [], []
for i in range(5):
    acc5, elapse5, newM = method5(M)
    accList.append(acc5), elapseList.append(elapse5), newMList.append(newM)
acc5, elapse5, newM = np.mean(accList), np.mean(elapseList), int(np.mean(newMList))
acc5, elapse5, newM

(0.94906, 2.6092892551422118, 9472)

In [71]:
accB, elapseB = baseline(newM)
accB, elapseB

(0.9484, 2.514045524597168)

In [72]:
acc1, elapse1, cluster_center_bucket, train_y_kmeans_bucket = method1(newM)
acc1, elapse1

(0.9703, 2.6321234703063965)

In [73]:
acc2, elapse2 = method2(newM, cluster_center_bucket, train_y_kmeans_bucket)
acc2, elapse2

(0.9567, 2.750431180000305)

In [74]:
acc3, elapse3, kmeans = method3(newM)
acc3, elapse3

(0.964, 2.5999537467956544)

In [75]:
acc4, elapse4 = method4(newM, kmeans)
acc4, elapse4

(0.9672, 2.6730650424957276)

In [77]:
def experiment(M):
    accB, elapseB = baseline(M)
    print("Baseline for M =", M, "acc:", accB, "time:", elapseB)
    acc1, elapse1, cluster_center_bucket, train_y_kmeans_bucket = method1(M)
    print("Method1 for M =", M, "acc:", acc1, "time:", elapse1)
    acc2, elapse2 = method2(M, cluster_center_bucket, train_y_kmeans_bucket)
    print("Method2 for M =", M, "acc:", acc2, "time:", elapse2)
    acc3, elapse3, kmeans = method3(M)
    print("Method3 for M =", M, "acc:", acc3, "time:", elapse3)
    acc4, elapse4 = method4(M, kmeans)
    print("Method4 for M =", M, "acc:", acc4, "time:", elapse4)
    accList, elapseList, newMList = [], [], []
    for i in range(5):
        acc5, elapse5, newM = method5(M)
        accList.append(acc5), elapseList.append(elapse5), newMList.append(newM)
    acc5, elapse5, newM = np.mean(accList), np.mean(elapseList), int(np.mean(newMList))
    print("Method5 for M =", newM, "acc:", acc5, "time:", elapse5)
    accB, elapseB = baseline(newM)
    print("Baseline for M =", newM, "acc:", accB, "time:", elapseB)
    acc1, elapse1, cluster_center_bucket, train_y_kmeans_bucket = method1(newM)
    print("Method1 for M =", newM, "acc:", acc1, "time:", elapse1)
    acc2, elapse2 = method2(newM, cluster_center_bucket, train_y_kmeans_bucket)
    print("Method2 for M =", newM, "acc:", acc2, "time:", elapse2)
    acc3, elapse3, kmeans = method3(newM)
    print("Method3 for M =", newM, "acc:", acc3, "time:", elapse3)
    acc4, elapse4 = method4(newM, kmeans)
    print("Method4 for M =", newM, "acc:", acc4, "time:", elapse4)

In [80]:
experiment(5000)

Baseline for M = 5000 acc: 0.9375 time: 1.455735206604004
Method1 for M = 5000 acc: 0.9701 time: 1.3934577465057374
Method2 for M = 5000 acc: 0.9497 time: 1.361889910697937
Method3 for M = 5000 acc: 0.9574 time: 1.3495391845703124
Method4 for M = 5000 acc: 0.9628 time: 1.4780715227127075
Method5 for M = 7466 acc: 0.94504 time: 1.8356473398208621
Baseline for M = 7466 acc: 0.945 time: 1.8568431615829468
Method1 for M = 7466 acc: 0.971 time: 1.8459997653961182
Method2 for M = 7466 acc: 0.9535 time: 1.9120234251022339
Method3 for M = 7466 acc: 0.961 time: 1.8280502080917358
Method4 for M = 7466 acc: 0.9662 time: 1.844463014602661


In [81]:
experiment(1000)

Baseline for M = 1000 acc: 0.877 time: 0.5445834159851074
Method1 for M = 1000 acc: 0.96 time: 0.5338339328765869
Method2 for M = 1000 acc: 0.9255 time: 0.5504796028137207
Method3 for M = 1000 acc: 0.9438 time: 0.5426963806152344
Method4 for M = 1000 acc: 0.952 time: 0.536484169960022
Method5 for M = 3667 acc: 0.93014 time: 1.066206388473511
Baseline for M = 3667 acc: 0.9299 time: 1.0822924613952636
Method1 for M = 3667 acc: 0.9664 time: 1.0578083515167236
Method2 for M = 3667 acc: 0.9457 time: 1.1423926830291748
Method3 for M = 3667 acc: 0.958 time: 1.151141381263733
Method4 for M = 3667 acc: 0.9634 time: 1.096830463409424


In [76]:
# Discussion: IVF methods used in FAISS does not throw away points. Rather, it limits the number of points evaluated by forming buckets first and only check neighboring buckets.